In [1]:
import os
import numpy as np
import pandas as pd


import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation as animation
import seaborn as sns
import ipywidgets as widgets

import scipy 
from scipy.signal import savgol_filter
import h5py

In [2]:
import sys
sys.path.append("../code/")
from extract_angles import get_tan_angles, get_bend_angles, savgol_skeleton
from extract_curvatures import get_curv_savgol,curvature_grad

In [3]:
sample_file = "/share/data/temp/athira/Tierpsy_skeleton_files_may15/20180706_105710_1_5m0s_None_None_None_skeletons.hdf5"
# sample_file = '/share/data/longterm/10/Marios Tierpsy analysis/30042020_120fps/analysis 120fps/Results/20200430_174745_1_0m30s_None_None_None_skeletons.hdf5'
skel_obj = h5py.File(sample_file, 'r')

In [4]:
skeleton_array = np.array(skel_obj['skeleton'])
contour1_array = np.array(skel_obj['contour_side1'])
contour2_array = np.array(skel_obj['contour_side2'])
qual = np.array(skel_obj['quality'])
mask = np.array(skel_obj['selection_mask'])
neck_ind_array = np.array(skel_obj['neck_point_array'])
skel_len_array = np.array(skel_obj['skeleton_length'])
contour_width_array = np.array(skel_obj['contour_width'])


In [5]:
# Apply NaN mask 
skeleton_array = skeleton_array[mask,:,:]
contour1_array = contour1_array[mask,:,:]
contour2_array = contour2_array[mask,:,:]
neck_ind_array = neck_ind_array[mask]
contour_width_array = contour_width_array[mask,:]
skeleton_array.shape

(7980, 49, 2)

In [6]:
slider_frame = widgets.IntSlider(value= 2951,
                                        min=0,
                                        max= skeleton_array.shape[0]-1,
                                        step=1,
                                        description='Frames:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')

c_pal = sns.color_palette("hls",9)

@widgets.interact
def plot_ciona_segments(ind = slider_frame):
    
    
    sample_skel = skeleton_array[ind,:,:] 
    sample_cont1 = contour1_array[ind,:,:] 
    sample_cont2 = contour2_array[ind,:,:] 
    
    fig, axes = plt.subplots(1,1,figsize=(5,5))

    axes.scatter(sample_skel[:,0],sample_skel[:,1], s=2)
    axes.plot(sample_cont1[:,0],sample_cont1[:,1])
    axes.plot(sample_cont2[:,0],sample_cont2[:,1])
    c12_x = np.concatenate((sample_cont1[:,0],np.flipud(sample_cont2[:,0])))
    c12_y = np.concatenate((sample_cont1[:,1],np.flipud(sample_cont2[:,1])))
    axes.fill(c12_x,c12_y, alpha= 0.7, label=skel_len_array[ind])
    axes.axis('equal')
    

interactive(children=(IntSlider(value=2951, continuous_update=False, description='Frames:', max=7979), Output(…

In [7]:
# Draw the ciona (TODO : Modify for segments)

skel_savgol = savgol_skeleton(skeleton_array)
cont1_savgol = savgol_skeleton(contour1_array)
cont2_savgol = savgol_skeleton(contour2_array)

cont1_savgol[:,0,:] = skel_savgol[:,0,:]
cont2_savgol[:,0,:] = skel_savgol[:,0,:]

cont1_savgol[:,-1,:] = skel_savgol[:,-1,:]
cont2_savgol[:,-1,:] = skel_savgol[:,-1,:]

In [8]:
def body_partitions_framewise(neck):
    
    segments_ind = {        
                'neck' : [(neck-1)/48, (neck+1)/48], 
                'head' : [0, (neck-1)/48],
#                 'head_tip' : [0, int(np.round((neck-1)/2))/48],
#                 'head_base' : [int(np.round((neck-1)/2))/48, (neck-1)/48],
                'tail': [(neck+1)/48 ,1],
                'tail_base': [(neck+1)/48 ,int(np.round((49-(neck+1))/5)+(neck+1))/48],
                'tail_pre_mid': [int(np.round((49-(neck+1))/5)+(neck+1))/48, int(np.round(2*(49-(neck+1))/5)+(neck+1))/48],
                'tail_mid': [ int(np.round(2*(49-(neck+1))/5)+(neck+1))/48,int(np.round(3*(49-(neck+1))/5)+(neck+1))/48],
                'tail_post_mid': [int(np.round(3*(49-(neck+1))/5)+(neck+1))/48, int(np.round(4*(49-(neck+1))/5)+(neck+1))/48],
                'tail_tip': [int(np.round(4*(49-(neck+1))/5)+(neck+1))/48,1]
            }
    
    return segments_ind

def get_segment_indices(contour_width,neck_point):
    
    max_skel_index = contour_width.shape[-1] - 1 # default : 48
    segments_ind_dflt = body_partitions_framewise(neck_point)
    segments_ind = { k: [int(round(x[0]*max_skel_index)),int(round(x[1]*max_skel_index))] for k,x in segments_ind_dflt.items()}
    return segments_ind

def segment_contours_fw(contour_1, contour_2, skel, segments_ind):
    
    contour_dict = {}
    
    for key, ind_bound in segments_ind.items():
        
        contour_1_x = contour_1[ind_bound[0]:ind_bound[1]+1,0]
        contour_2_x = contour_2[ind_bound[0]:ind_bound[1]+1,0]
        contour_2_x = np.flipud(contour_2_x)
        
        contour_1_y = contour_1[ind_bound[0]:ind_bound[1]+1,1]
        contour_2_y = contour_2[ind_bound[0]:ind_bound[1]+1,1]
        contour_2_y = np.flipud(contour_2_y)
        
        contour_x = np.concatenate((contour_1_x, contour_2_x))
        contour_y = np.concatenate((contour_1_y, contour_2_y))
        contour_dict[key] = np.array([contour_x, contour_y])
        
    return contour_dict

def calc_centroids_fw(segments_ind, neck_point):
    centroid_dict = {}
    
    for key, value in segments_ind.items():
        
        centroid_dict[key] = round((value[0] + value[1])/2)
    
    centroid_dict['neck'] = neck_point
#     centroid_dict['tail'] = segments_ind['tail_mid_distal'][0]
#     centroid_dict['head'] = segments_ind['head_base'][0]
    
    centroid_dict['tail'] = centroid_dict['tail_mid']
    #centroid_dict['head'] = segments_ind['head_base'][0]
    return centroid_dict

In [9]:
slider_frame = widgets.IntSlider(value= 1780,
                                        min=0,
                                        max= skeleton_array.shape[0]-1,
                                        step=1,
                                        description='Frames:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')

slider_skel_1 = widgets.IntSlider(value= 15,
                                        min=0,
                                        max= skeleton_array.shape[1]-1,
                                        step=1,
                                        description='Point 1:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')

slider_skel_2 = widgets.IntSlider(value= 40,
                                        min=0,
                                        max= skeleton_array.shape[1]-1,
                                        step=1,
                                        description='Point 2:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')




In [10]:
@widgets.interact
def illustrate_tan(ind = slider_frame, skel_point_1 = slider_skel_1, skel_point_2 = slider_skel_2 ):
    
    plt.close()
    
    xData = skel_savgol[ind,:,0]
    yData = skel_savgol[ind,:,1]
    
    fig, axes = plt.subplots(1,1,figsize=(5,5), dpi=150)
    c_pal = sns.color_palette("hls",9)

    ############ Colouring Segments
    
    segments_ind = get_segment_indices(contour_width_array[ind],neck_ind_array[ind])
    contour_dict = segment_contours_fw(cont1_savgol[ind], cont2_savgol[ind],skel_savgol[ind], segments_ind)
    
    color_dict = {}
    for key, color in zip(contour_dict.keys(), c_pal):
        color_dict[key] = color
    
    axes.scatter(skel_savgol[ind,:,0], skel_savgol[ind,:,1], c= 'k',s=2,alpha=0.5)
    for key, val in contour_dict.items():
        if key not in ['tail']:
            axes.fill(contour_dict[key][0],contour_dict[key][1],label=key, color = color_dict[key], alpha=0.5)
            
        
        if skel_point_1 in range(segments_ind[key][0],segments_ind[key][1]):
            color_1 = color_dict[key]
        if skel_point_2 in range(segments_ind[key][0],segments_ind[key][1]):
            color_2 = color_dict[key]
    
    
    ############ Drawing line segments
        if key in ['head','tail_mid']:
            axes.plot([skel_savgol[ind,segments_ind[key][0],0],skel_savgol[ind,segments_ind[key][1],0]],
                     [skel_savgol[ind,segments_ind[key][0],1],skel_savgol[ind,segments_ind[key][1],1]],
                     c = color_dict[key] )
    
    
    axes.plot()
    
    
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)

    axes.axis('equal')
    
    fig.savefig('../figures1/angles_definition_a.svg')

interactive(children=(IntSlider(value=1780, continuous_update=False, description='Frames:', max=7979), IntSlid…

In [11]:
def translate(source, target_x, target_y):
    
    diff_x = target_x - source[0][0]
    diff_y = target_y - source[1][0]
    linesegment_trans_x = source[0] + diff_x
    linesegment_trans_y = source[1] + diff_y
    
    return linesegment_trans_x, linesegment_trans_y

In [12]:
def rotate_matrix(skel_array,angle):
    xr = skel_array[:,0]*np.cos(angle) - skel_array[:,1]* np.sin(angle)
    yr = skel_array[:,0]*np.sin(angle) + skel_array[:,1]* np.cos(angle)
    skel_array_r = np.empty_like(skel_array)
    
    skel_array_r[:,0] = xr
    skel_array_r[:,1] = yr
    return skel_array_r

In [13]:
slider_frame = widgets.IntSlider(value= 1788,
                                        min=0,
                                        max= skeleton_array.shape[0]-1,
                                        step=1,
                                        description='Frames:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d')

# slider_skel_1 = widgets.IntSlider(value= 15,
#                                         min=0,
#                                         max= skeleton_array.shape[1]-1,
#                                         step=1,
#                                         description='Point 1:',
#                                         disabled=False,
#                                         continuous_update=False,
#                                         orientation='horizontal',
#                                         readout=True,
#                                         readout_format='d')

slider_segment = widgets.IntRangeSlider(value=[22,29],
                                        min=0,
                                        max=skeleton_array.shape[1]-1,
                                        step=1,
                                        description='Test:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='d',
                                    )

slider_angle = widgets.FloatSlider(value= 0.75,
                                        min=-np.pi,
                                        max= np.pi,
                                        step=np.pi/180,
                                        description='Angle:',
                                        disabled=False,
                                        continuous_update=False,
                                        orientation='horizontal',
                                        readout=True,
                                        readout_format='1.2f')






In [14]:
def extended(x1,x2,y1,y2,x1_new, x2_new):


    x_ext = np.linspace(x1_new, x2_new, 200)  
    p = np.polyfit([x1,x2], [y1,y2], deg=1)
    y_ext = np.poly1d(p)(x_ext)
    
    return [x_ext,y_ext]

In [25]:
def get_angle_plot_simple(line1, line2, c = None, origin = [0,0]):
    

    import math
     

    # Angle between line1 and x-axis
    slope1 = (line1[1][-1] - line1[1][0]) / float(line1[0][-1] - line1[0][0])
    angle1 = abs(math.degrees(math.atan(slope1))) # Taking only the positive angle


    # Angle between line2 and x-axis
    slope2 = (line2[1][-1] - line2[1][0]) / float(line2[0][-1] - line2[0][0])
    angle2 = abs(math.degrees(math.atan(slope2)))

    theta1 = min(angle1, angle2)
    theta2 = max(angle1, angle2)

    angle = theta2 - theta1
    
    arc_head = matplotlib.patches.Arc((origin[0],origin[1]), width = 30, height = 30, angle= 0,
                                        theta1=0, theta2=theta2, color =c, zorder=2, lw = 3)
    wedge_head = matplotlib.patches.Wedge((origin[0],origin[1]), theta1=0, theta2=theta2, 
                                          r = 15, color =c, zorder=1, alpha =0.5)
    

    return [wedge_head, arc_head]

In [26]:
def get_angle_plot(line1, line2, c_head = None, c_seg = None, c_rel = None, origin = [0,0]):
    

    import math
     

    # Angle between line1 and x-axis
    slope1 = (line1[1][-1] - line1[1][0]) / float(line1[0][-1] - line1[0][0])
    angle1 = abs(math.degrees(math.atan(slope1))) # Taking only the positive angle


    # Angle between line2 and x-axis
    slope2 = (line2[1][-1] - line2[1][0]) / float(line2[0][-1] - line2[0][0])
    angle2 = abs(math.degrees(math.atan(slope2)))

    theta1 = min(angle1, angle2)
    theta2 = max(angle1, angle2)

    angle = theta2 - theta1
    print(angle, theta1, theta2)
    arc_diff = matplotlib.patches.Arc((line2[0][0],line2[1][0]), width = 35, height = 35, angle= 0,
                                        theta1=theta1, theta2=theta2, color = c_rel, lw = 10)
    arc_diff_border = matplotlib.patches.Arc((line2[0][0],line2[1][0]), width = 37, height = 37, angle= 0,
                                        theta1=theta1, theta2=theta2, color = c_rel, lw = 3)
    
    arc_head = matplotlib.patches.Arc((line2[0][0],line2[1][0]), width = 30, height = 30, angle= 0,
                                        theta1=0, theta2=theta1, color =c_head, zorder=2, lw = 3)
    wedge_head = matplotlib.patches.Wedge((line2[0][0],line2[1][0]), theta1=0, theta2=theta1, 
                                          r = 15, color =c_head, zorder=1, alpha =0.5)
    
    
    arc_seg = matplotlib.patches.Arc((line2[0][0],line2[1][0]), width = 20, height = 20, angle= 0,
                                       theta1=0, theta2=theta2, color = c_seg, zorder=1, lw = 3)
    wedge_seg = matplotlib.patches.Wedge((line2[0][0],line2[1][0]), theta1=0, theta2=theta2, 
                                          r = 10, color = c_seg, zorder=2, alpha =0.5)

    return [arc_diff, arc_diff_border, arc_head, arc_seg, wedge_head, wedge_seg]

In [95]:

def circarrow(ax,diameter,centX,centY,startangle,angle,startarrow=False, endarrow=False,**kwargs):

#     startarrow=kwargs.pop("startarrow",False)
#     endarrow=kwargs.pop("endarrow",False)

    arc = matplotlib.patches.Arc([centX,centY],diameter,diameter,angle=startangle,
          theta1=np.rad2deg(kwargs.get("head_length",1.5*3*.001)) if startarrow else 0,
                                 theta2=angle-(np.rad2deg(kwargs.get("head_length",1.5*3*.001)) if endarrow else 0),
                                 linestyle="-",color=kwargs.get("color","black"), zorder=3)
#     ax.add_patch(arc)

    if startarrow:
        startX=diameter/2*np.cos(np.radians(startangle))
        startY=diameter/2*np.sin(np.radians(startangle))
        startDX=+.000001*diameter/2*np.sin(np.radians(startangle)+kwargs.get("head_length",1.5*3*.001))
        startDY=-.000001*diameter/2*np.cos(np.radians(startangle)+kwargs.get("head_length",1.5*3*.001))
        arr1 = plt.arrow(startX-startDX,startY-startDY,startDX,startDY,**kwargs)
#         ax.add_patch(arr1)

    if endarrow:
        endX=diameter/2*np.cos(np.radians(startangle+angle))
        endY=diameter/2*np.sin(np.radians(startangle+angle))
        endDX=-.000001*diameter/2*np.sin(np.radians(startangle+angle)-kwargs.get("head_length",1.5*3*.001))
        endDY=+.000001*diameter/2*np.cos(np.radians(startangle+angle)-kwargs.get("head_length",1.5*3*.001))
        arr2 = plt.arrow(endX-endDX,endY-endDY,endDX,endDY,**kwargs)
#         ax.add_patch(arr2)
    
    return [arc,arr2]

In [96]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False

In [100]:
@widgets.interact
def illustrate_tan_rotate(ind = slider_frame, skel_points = slider_segment, angle = slider_angle,
                         ):
    
    plt.close()
    
    skel_savgol_r = rotate_matrix(skel_savgol[ind],angle)
    cont1_savgol_r = rotate_matrix(cont1_savgol[ind],angle)
    cont2_savgol_r = rotate_matrix(cont2_savgol[ind],angle)
    
    xData = skel_savgol_r[:,0]
    yData = skel_savgol_r[:,1]
    
    fig, axes = plt.subplots(1,1,figsize=(10,10), dpi=100)
    c_pal = sns.color_palette("hls",9)
    c_pal_sat = sns.color_palette("husl", 9)#sns.hls_palette(9, l=.5, s=.9)
    
    
    ############ Colouring Segments
    
    segments_ind = get_segment_indices(contour_width_array[ind],neck_ind_array[ind])
    contour_dict = segment_contours_fw(cont1_savgol_r, cont2_savgol_r, skel_savgol_r, segments_ind)
    
    color_dict = {}
    color_dict_sat = {}
    for key, color, color_sat in zip(contour_dict.keys(), c_pal, c_pal_sat):
        color_dict[key] = color
        color_dict_sat[key] = color_sat
    
    axes.scatter(skel_savgol_r[:,0], skel_savgol_r[:,1], c= 'k',s=2,alpha=0.3)
    axes.plot(skel_savgol_r[:,0], skel_savgol_r[:,1], c= 'k',alpha=0.3)
    axes.plot(cont1_savgol_r[:,0], cont1_savgol_r[:,1], c='k', alpha=0.5)
    axes.plot(cont2_savgol_r[:,0], cont2_savgol_r[:,1], c='k', alpha=0.5)
    
    
    for key, val in contour_dict.items():
        if key not in ['tail']:
            axes.fill(contour_dict[key][0],contour_dict[key][1],label=key, color = color_dict[key], alpha=0.5)
                
        if (skel_points[0]+skel_points[1])//2 in range(segments_ind[key][0],segments_ind[key][1]):
            axes.plot([skel_savgol_r[skel_points[0],0],skel_savgol_r[skel_points[1],0]],
                     [skel_savgol_r[skel_points[0],1],skel_savgol_r[skel_points[1],1]],
                     c = color_dict_sat[key],lw=2 )
            axes.scatter([skel_savgol_r[skel_points[0],0],skel_savgol_r[skel_points[1],0]],
                     [skel_savgol_r[skel_points[0],1],skel_savgol_r[skel_points[1],1]],
                     c = 'k',s=2, zorder=3)
            segment_key = key
            
            
            
    ############ Drawing line segments
        if key in ['head']:
            axes.plot([skel_savgol_r[segments_ind[key][0],0],skel_savgol_r[segments_ind[key][1],0]],
                     [skel_savgol_r[segments_ind[key][0],1],skel_savgol_r[segments_ind[key][1],1]],
                     c = color_dict_sat[key] ,lw=2, zorder=2)
            axes.scatter([skel_savgol_r[segments_ind[key][0],0],skel_savgol_r[segments_ind[key][1],0]],
                     [skel_savgol_r[segments_ind[key][0],1],skel_savgol_r[segments_ind[key][1],1]],
                     c = 'k',s=2, zorder=3)
            color_head = color_dict[key]
            
            
    
    xlim = axes.get_xlim()
    ylim = axes.get_ylim()
#     axes.plot([xlim[0]-40,xlim[1]+10],
#              [ylim[0]-5,ylim[0]-5],lw=1)
    
    ref_axis = extended(xlim[0]-40,xlim[1]+10,ylim[0]-5,ylim[0]-5,xlim[0]-40,xlim[1]+10)
    axes.plot(ref_axis[0],ref_axis[1],lw=1, zorder=3, ls=':')
    
    
    for key in ['head', segment_key]:
        
        if key == 'head':
            x_limit_lower = -35
            ext_coords_head = extended(skel_savgol_r[segments_ind[key][0],0],skel_savgol_r[segments_ind[key][1],0],
                                           skel_savgol_r[segments_ind[key][0],1],skel_savgol_r[segments_ind[key][1],1],
                                           xlim[0]+x_limit_lower,xlim[1]+10)
            axes.plot(ext_coords_head[0],ext_coords_head[1],ls=':',c=color_dict_sat[key],zorder=1)   
        else:
            x_limit_lower = 15
            ext_coords = extended(skel_savgol_r[segments_ind[key][0],0],skel_savgol_r[segments_ind[key][1],0],
                                           skel_savgol_r[segments_ind[key][0],1],skel_savgol_r[segments_ind[key][1],1],
                                           xlim[0]+x_limit_lower,xlim[1]+10)


            axes.plot(ext_coords[0],ext_coords[1],ls=':',c=color_dict_sat[key],zorder =1)

#             print(ref_axis[0],ext_coords[0])
#             idx = np.argwhere(np.diff(np.sign(ref_axis[1] - ext_coords[1]))).flatten()
            
            L1 = line([ref_axis[0][0],ref_axis[1][0]], [ref_axis[0][-1],ref_axis[1][-1]])
            L2 = line([ext_coords[0][0],ext_coords[1][0]], [ext_coords[0][-1],ext_coords[1][-1]])
            L3 = line([ext_coords_head[0][0],ext_coords_head[1][0]], [ext_coords_head[0][-1],ext_coords_head[1][-1]])

            R = intersection(L1, L2)
            R_head = intersection(L1, L3)
            
            ext_coords_trans = translate(ext_coords_head, R[0],R[1])
            axes.plot(ext_coords_trans[0],ext_coords_trans[1],ls=':',c = color_dict_sat['head'],zorder =1)
#             axes.scatter(ext_coords[0][idx], ext_coords[1][idx], zorder =2, s=4)
            axes.scatter(R[0], R[1], c = color_dict_sat[segment_key], zorder =2, s=7)
            axes.scatter(R_head[0], R_head[1], c = color_dict_sat['head'], zorder =2, s=7)
            
    arc_artists = get_angle_plot(ext_coords, ext_coords_trans, c_head = color_dict['head'], 
                                c_seg = color_dict[segment_key],
                                c_rel = color_dict_sat[segment_key],
                                origin = [R[0],R[1]])
    
    arc_artists_1 = get_angle_plot_simple(ref_axis, ext_coords_head, c = color_head, origin = R_head)

#     arc_artists_2 = circarrow(axes,30,R_head[0],R_head[1],startangle=0,angle = 1,startarrow=True,endarrow=True,width=10,head_width=10,
#                   head_length=10,length_includes_head=False,color="black", zorder=3)
    for arc_artist in arc_artists:
        axes.add_artist(arc_artist)
    for arc_artist in arc_artists_1:
        axes.add_artist(arc_artist)
#     for arc_artist in arc_artists_2:
#         axes.add_artist(arc_artist)
   
    
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)

    axes.axis('equal')

    
    fig.savefig('../figures1/angles_definition_b.svg')

interactive(children=(IntSlider(value=1788, continuous_update=False, description='Frames:', max=7979), IntRang…